In [1]:
import pandas as pd
import multiprocessing as mp
import numpy as np
logs = pd.read_csv("./data_nov/logs_nov.csv")
proc = pd.read_csv("./data_nov/processes_nov.csv")
orders = pd.read_csv("./data_nov/orders_nov.csv")
# .parquet
logs.time_out = pd.to_datetime(logs.time_out, infer_datetime_format=True)
orders.set_index('order_number', inplace=True)

In [4]:
orders

,customer_job_number,reference_number,item_number,item_description,product_family,product_model,product_variant,product_clearance,product_length,product_block_count,...,order_release_code,order_customer_name,order_replied_due,order_scheduled_due,order_original_due,order_scheduled_shipping_date,order_class,promise_date,sales_item_number,printed_due
order_number,,,,,,,,,,,,,,,,,,,,,
M057310,NaN,GY4,41381520001000A,SRS15W-1000LM HALF RAIL 4,SRS-W,15,NaN,NaN,1000.0,0,...,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-02-18
M860258,NaN,HK168605,414010100004000,SRS9X-400LM RAIL,SRS,9X,NaN,NaN,400.0,0,...,5,NaN,2021-12-08,2022-04-15,2021-12-08,2021-12-08,21.0,NaN,NaN,NaN
M057311,NaN,GY4,41381520001000A,SRS15W-1000LM HALF RAIL 4,SRS-W,15,NaN,NaN,1000.0,0,...,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-02-18
M049427,NaN,SS02G,875050320002001,GL20N TABLE X200L RD A6963,NaN,NaN,NaN,NaN,NaN,1,...,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-01-11
M049428,NaN,SS02G,875054800010000,GL COVER - X1000,NaN,NaN,NaN,NaN,NaN,1,...,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-01-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
M168316,NaN,GY3,301E66222009000,HSR65XLC HALF BLOCK 3,HSR,65,XLC,NaN,NaN,1,...,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-18
M168425,RM123883GR,HK264841,502451011624401,SHS20-340LF-II RAIL(MOTION),SHS,20,NaN,NaN,340.0,0,...,5,MOTION SOLUTIONS,2022-12-08,2022-12-08,2022-11-28,2022-12-08,11.0,NaN,102351011624400,2022-10-11
M168426,RM138399GR,HK277417,202A45A22200000,HSR45C1SSC1(GK) BLOCK,HSR,45,C,C1,NaN,1,...,5,NaN,2022-12-02,2022-12-02,2022-12-02,2022-12-02,21.0,NaN,202A45A22200000,2022-12-02


In [5]:
vip_o = orders[(~orders.product_length.isnull()) & (orders.order_status == 55) & (orders.reference_number.str.startswith('HK') | (orders.reference_number == 'GYHAAS')) & (orders.item_description.str.contains('RAIL'))]
vip_orders = vip_o.copy()
vip_orders['y'] = 0
vip_orders['steps'] = 0
# vip_orders['start'] = ''
vip_mos = vip_orders.index.values.tolist()
vip_logs = logs[(logs.order_number.isin(vip_mos)) & (logs.quantity_complete != 0)]
vip_logs.sort_values('sequence_code', ascending=True, inplace=True)
drop_cols = ['customer_job_number', 'reference_number', 'item_number', 'item_description',  'product_variant', 'product_clearance',  'product_block_count', 'product_g1', 'product_pitch',  'order_status', 'order_release_code', 'order_customer_name', 'order_replied_due', 'order_scheduled_due', 'order_original_due', 'order_scheduled_shipping_date', 'order_class', 'promise_date', 'sales_item_number', 'printed_due']
vip_orders = vip_orders.drop(columns=drop_cols)

/tmp/ipykernel_957/699425637.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vip_logs.sort_values('sequence_code', ascending=True, inplace=True)


In [130]:
logs.columns
# logs.pivot(index='order_number', columns='name', values='values')


Index(['order_number', 'employee_number', 'quantity_complete',
       'quantity_scrapped', 'facility_id', 'machine_id', 'sequence_code',
       'time_in', 'time_out', 'employee_name', 'employee_first_name',
       'employee_last_name'],
      dtype='object')

In [ ]:
# test = logs.pivot(index='order_number', columns='facility_id', values='time_out')
# mo_logs = mo_logs.drop_duplicates(subset=['sequence_code'], keep='first')
# test


In [6]:
THROTTLE_MIN = 1800
def f(mo):       
    mo_logs = vip_logs[vip_logs.order_number == mo].copy()
    mo_logs = mo_logs.drop_duplicates(subset=['sequence_code'], keep='first')    
    print(mo_logs.columns)
    mo_logs['diff'] = mo_logs['time_out'].diff(periods=1).astype('timedelta64[m]')
    print(mo_logs.columns)
    # mo_logs['start'] = mo_logs['time_out'].shift(1)
    mo_logs.loc[mo_logs['diff'] > THROTTLE_MIN, 'diff'] = THROTTLE_MIN
    test = mo_logs.pivot(index='order_number', columns='facility_id', values='diff')
    print(test)
    try:
        # for index, row in mo_logs.iloc[1:].iterrows():                
        #     if row['diff'] > THROTTLE_MIN:
        #         end = row.time_out.strftime("%Y-%m-%d")
        #         start = row.start.strftime("%Y-%m-%d")
        #         offday_count = len(list(x for x in low_days if start < x < end))             
        #         row['diff'] -= offday_count * 1440
        #         if row['diff'] > THROTTLE_MIN:
        #             # print(index)
        #             mo_logs.loc[index, 'diff'] = THROTTLE_MIN    
        period = mo_logs['diff'].sum() / 60
        # return mo, period, mo_logs.iloc[0]['time_out']
        return mo, period, len(mo_logs)
    except Exception as e: print(mo, e)
        
def append_res(result):        
    for i in result:
        if i:
            mo, period, steps = i                            
            try:                
                vip_orders.loc[mo, 'y'] = period
                vip_orders.loc[mo, 'steps'] = steps
            except Exception as e: print(e)

if __name__ == "__main__":        
    with mp.Pool(16) as pool:        
        print('pool started')
        # results = pool.map_async(f, vip_mos[:100], callback = append_res)                        
        results = pool.map_async(f, vip_mos[:2])
        results.wait() 
print("job_done")

       order_number  employee_number  quantity_complete  quantity_scrapped  \
633515      M860258              699                 20                  0   
631488      M860258             1672                 20                  0   
630101      M860258              708                 20                  0   

       facility_id machine_id  sequence_code                 time_in  \
633515       D0010        NaN             10  2021-11-22 19:18:37+00   
631488       D0020     SGH017             20  2021-11-23 16:22:36+00   
630101       F0010     ZWH001             30  2021-11-24 10:31:06+00   

                        time_out     employee_name employee_first_name  \
633515 2021-11-22 19:26:18+00:00  Grether, Richard             Grether   
631488 2021-11-23 16:23:12+00:00      Goble, Brian               Goble   
630101 2021-11-24 10:31:12+00:00  Kinser, Derek A.              Kinser   

       employee_last_name  
633515            Richard  
631488              Brian  
630101           

In [ ]:
b_types = ['-III', '(GK)', '(GP)', '-II']
p_types = ['K','E','P','F','T','S','M','H']
vip_orders = vip_orders[vip_orders['y'] > 6]
des = vip_orders['item_description']

vip_orders['LE'] = np.where(vip_orders['item_description'].str.count('LE'), 1, 0)
vip_orders['GK'] = np.where(vip_orders['item_description'].str.count('GK'), 1, 0)
vip_orders['LT'] = np.where(vip_orders['item_description'].str.count('LT'), 1, 0)
vip_orders['GP'] = np.where(vip_orders['item_description'].str.count('GP'), 1, 0)

In [ ]:
b_types = ['-III', '(GK)', '(GP)', '-II']
p_types = ['K','E','P','F','T','S','M','H']
for i in b_types:
    vip_orders[i] = 0
for i in p_types:
    vip_orders[i] = 0
vip_orders['custom'] = 0
for index, row in vip_orders.iterrows():
    try:
        des = row['item_description'].split(' RAIL')[0].split('L')[1]
        for bt in b_types:
            if bt in des:
                vip_orders.loc[index, bt] = 1
                des = des.replace(bt, '')
        print(des)
        for pt in p_types:
            if pt in des:            
                vip_orders.loc[index, pt] = 1
    except Exception as e:
        vip_orders.loc[index, 'custom'] = 1
vip_orders      




In [ ]:
vip_orders.columns.tolist()
drop_cols = ['customer_job_number',
 'reference_number',
 'item_number',
 'item_description', 
 'product_variant',
 'product_clearance', 
 'product_block_count',
 'product_g1',
 'product_pitch', 
 'order_status',
 'order_release_code',
 'order_customer_name',
 'order_replied_due',
 'order_scheduled_due',
 'order_original_due',
 'order_scheduled_shipping_date',
 'order_class',
 'promise_date',
 'sales_item_number',
 'printed_due',
 ]
test = vip_orders.drop(columns=drop_cols)



In [ ]:
test.columns

In [116]:
cols = ['product_family', 'product_model', 'product_length', 'order_quantity',
       'LE', 'GK', 'LT', 'GP', '-III', '(GK)', '(GP)', '-II',
       'K', 'E', 'P', 'F', 'T', 'S', 'M', 'H', 'custom', 'steps', 'y']
test = test[cols]
test.loc[test.y < 8, 'y'] = 8

In [115]:
test.describe()
# test.to_csv('nov_vip_mo_throttled_24h.csv')

,product_family,product_model,product_length,order_quantity,LE,GK,LT,GP,-III,(GK),...,E,P,F,T,S,M,H,custom,steps,y
count,42716.000000,42716.000000,42716.000000,42716.000000,42716.000000,42716.000000,42716.000000,42716.000000,42716.000000,42716.000000,...,42716.000000,42716.000000,42716.000000,42716.000000,42716.000000,42716.000000,42716.000000,42716.000000,42716.000000,42716.000000
mean,3.129834,5.572736,1.200869,8.274628,0.036029,0.373233,0.029825,0.126580,0.004003,0.373233,...,0.040477,0.049162,0.043473,0.031464,0.015966,0.199527,0.016926,0.000702,6.370189,90.145449
std,1.661920,3.399888,1.322435,5.158663,0.186364,0.483669,0.170106,0.332506,0.063145,0.483669,...,0.197077,0.216208,0.203922,0.174569,0.125345,0.399650,0.128995,0.026492,2.583451,48.875940
min,0.000000,0.000000,0.038000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,1.150000
25%,2.000000,3.000000,0.340000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,55.262500
50%,3.000000,4.000000,0.760000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,82.800000
75%,4.000000,9.000000,1.568000,10.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,113.366667
max,9.000000,16.000000,56.290000,50.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,19.000000,323.000000


In [124]:
list_classify = ["product_family", "product_model"]
for i in list_classify:
    test[i] = pd.factorize(test[i])[0] + 1  
test['product_length'] = test['product_length'] / 1000
classfied = test[test.product_family.isin(range(2, 4))]
pivot = int(len(classfied) * .2)
shuffled = classfied.sample(frac=1)
train = shuffled.iloc[pivot:].copy()
val = shuffled.iloc[:pivot].copy()
train = train.astype(float)
val = val.astype(float)
train.to_csv('train_throttled_80_mo_24h.csv')
val.to_csv('val_throttled_80_mo_24h.csv')

In [126]:
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

class MyDataset(Dataset): 
  def __init__(self,data):     
    super(MyDataset).__init__()
    x=data.iloc[:,0:22].values
    y=data.iloc[:,22].values     
    self.x_train=torch.tensor(x,dtype=torch.float32).cuda()    
    self.y_train=torch.tensor(y,dtype=torch.float32).view(-1, 1).cuda()    
 
  def __len__(self):
    return len(self.y_train)   
  def __getitem__(self,idx):
    return self.x_train[idx],self.y_train[idx]

train_set=MyDataset(train)
val_set=MyDataset(val)
train_dl=DataLoader(train_set, batch_size=256, shuffle=True, num_workers=0)
val_dl=DataLoader(val_set,batch_size=len(val_set))

In [128]:
loss_fn = F.mse_loss
def fit(num_epochs, model, loss_fn, opt):
    val_best  = 23
    for epoch in range(num_epochs):        
        for xb,yb in train_dl:            
            pred = model(xb)
            loss = loss_fn(pred, yb)
            loss.backward()
            opt.step()
            opt.zero_grad()
        if epoch % 50 == 0:
            loss_val = loss.item()
            print(loss_val, epoch)            
            for v_xb,v_yb in val_dl:
                result = model(v_xb)     
                val_err = torch.mean(abs(v_yb - result) / v_yb).item() * 100    
                print("val_err: ", val_err)
            if val_err < val_best:
                val_best = val_err
                torch.save(model.state_dict(), f"throttle_{str(val_err).replace('.', '_')[:5]}.pth")
    
class SimpleNet(nn.Module):    
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(22, 128)        
        self.linear2 = nn.Linear(128, 64)        
        self.linear3 = nn.Linear(64, 32)        
        self.linear4 = nn.Linear(32, 1)
        
        self.act1 = nn.ReLU() # Activation function        
        self.act2 = nn.ReLU() # Activation function
        self.act3 = nn.ReLU() # Activation function
            
    # Perform the computation
    def forward(self, x):                
        x = self.linear1(x)
        x = self.act1(x)
        x = self.linear2(x)
        x = self.act2(x)
        x = self.linear3(x)        
        x = self.act3(x)
        x = self.linear4(x)        
        return x

model = SimpleNet().cuda()
opt = torch.optim.SGD(model.parameters(), 1e-6, momentum=0.9)
fit(1000, model, loss_fn, opt)

10592.3330078125 0
val_err:  97.70854711532593
762.1976318359375 50
val_err:  24.34912770986557
612.4300537109375 100
val_err:  23.257488012313843
623.1239013671875 150
val_err:  23.293687403202057
667.3626708984375 200
val_err:  23.333217203617096
618.1187744140625 250
val_err:  23.23050945997238
632.7777099609375 300
val_err:  22.6587176322937
516.1515502929688 350
val_err:  23.135297000408173
609.0120239257812 400
val_err:  23.163023591041565
629.2217407226562 450
val_err:  22.6850226521492
472.64544677734375 500
val_err:  22.772029042243958
606.0136108398438 550
val_err:  23.24056327342987
516.6710205078125 600
val_err:  23.43589961528778
526.7083740234375 650
val_err:  23.230120539665222
595.5048828125 700
val_err:  23.02795797586441
553.197509765625 750
val_err:  23.065121471881866
536.4781494140625 800
val_err:  22.970521450042725
470.1539001464844 850
val_err:  23.083800077438354
472.9566955566406 900
val_err:  23.199351131916046
527.3587646484375 950
val_err:  22.2085773944854

In [120]:
len(test)

42716

In [ ]:
# low_days = ['2021-07-03', '2021-07-04', '2021-07-10', '2021-07-11', '2021-07-17', '2021-07-18', '2021-07-24', '2021-07-25', '2021-07-31', '2021-08-01', '2021-08-07', '2021-08-08', '2021-08-14', '2021-08-15', '2021-08-21', '2021-08-22', '2021-08-28', '2021-08-29', '2021-09-04', '2021-09-05', '2021-09-11', '2021-09-12', '2021-09-18', '2021-09-19', '2021-09-25', '2021-09-26', '2021-10-02', '2021-10-03', '2021-10-09', '2021-10-10', '2021-10-16', '2021-10-17', '2021-10-23', '2021-10-24', '2021-10-30', '2021-10-31', '2021-11-06', '2021-11-07', '2021-11-13', '2021-11-14', '2021-11-20', '2021-11-21', '2021-11-27', '2021-11-28', '2021-12-04', '2021-12-05', '2021-12-11', '2021-12-12', '2021-12-18', '2021-12-19', '2021-12-25', '2021-12-26', '2021/06/05', '2021/06/06', '2021/06/13', '2021/06/19', '2021/06/20', '2021/06/27', '2021/06/30', '2021/07/02', '2021/07/03', '2021/07/04', '2021/07/05', '2021/07/11', '2021/07/17', '2021/07/18', '2021/07/25', '2021/07/31', '2021/08/01', '2021/08/08', '2021/08/14', '2021/08/15', '2021/08/22', '2021/08/28', '2021/08/29', '2021/09/04', '2021/09/05', '2021/09/06', '2021/09/12', '2021/09/18', '2021/09/19', '2021/09/26', '2021/10/02', '2021/10/03', '2021/10/10', '2021/10/16', '2021/10/17', '2021/10/24', '2021/10/30', '2021/10/31', '2021/11/07', '2021/11/13', '2021/11/14', '2021/11/21', '2021/11/25', '2021/11/26', '2021/11/27', '2021/11/28', '2021/12/05', '2021/12/11', '2021/12/12', '2021/12/19', '2021/12/23', '2021/12/24', '2021/12/25', '2021/12/26', '2021/12/31', '2022-01-01', '2022-01-02', '2022-01-08', '2022-01-09', '2022-01-15', '2022-01-16', '2022-01-22', '2022-01-23', '2022-01-29', '2022-01-30', '2022-02-05', '2022-02-06', '2022-02-12', '2022-02-13', '2022-02-19', '2022-02-20', '2022-02-26', '2022-02-27', '2022-03-05', '2022-03-06', '2022-03-12', '2022-03-13', '2022-03-19', '2022-03-20', '2022-03-26', '2022-03-27', '2022-04-02', '2022-04-03', '2022-04-09', '2022-04-10', '2022-04-16', '2022-04-17', '2022-04-23', '2022-04-24', '2022-04-30', '2022-05-01', '2022-05-07', '2022-05-08', '2022-05-14', '2022-05-15', '2022-05-21', '2022-05-22', '2022-05-28', '2022-05-29', '2022-06-04', '2022-06-05', '2022-06-11', '2022-06-12', '2022-06-18', '2022-06-19', '2022-06-25', '2022-06-26', '2022-07-02', '2022-07-03', '2022-07-09', '2022-07-10', '2022-07-16', '2022-07-17', '2022-07-23', '2022-07-24', '2022-07-30', '2022-07-31', '2022-08-06', '2022-08-07', '2022-08-13', '2022-08-14', '2022-08-20', '2022-08-21', '2022-08-27', '2022-08-28', '2022-09-03', '2022-09-04', '2022-09-10', '2022-09-11', '2022-09-17', '2022-09-18', '2022-09-24', '2022-09-25', '2022-10-01', '2022-10-02', '2022-10-08', '2022-10-09', '2022-10-15', '2022-10-16', '2022-10-22', '2022-10-23', '2022-10-29', '2022-10-30', '2022-11-05', '2022-11-06', '2022/01/01', '2022/01/02', '2022/01/03', '2022/01/08', '2022/01/09', '2022/01/16', '2022/01/22', '2022/01/23', '2022/01/30', '2022/02/04', '2022/02/05', '2022/02/06', '2022/02/13', '2022/02/19', '2022/02/20', '2022/02/27', '2022/03/05', '2022/03/06', '2022/03/12', '2022/03/13', '2022/03/19', '2022/03/20', '2022/03/27', '2022/04/02', '2022/04/03', '2022/04/10', '2022/04/15', '2022/04/16', '2022/04/17', '2022/04/23', '2022/04/24', '2022/04/30', '2022/05/01', '2022/05/08', '2022/05/14', '2022/05/15', '2022/05/22', '2022/05/28', '2022/05/29', '2022/05/30', '2022/06/05', '2022/06/11', '2022/06/12', '2022/06/18', '2022/06/19', '2022/06/25', '2022/06/26', '2022/06/30', '2022/07/02', '2022/07/03', '2022/07/04', '2022/07/05', '2022/07/10', '2022/07/16', '2022/07/17', '2022/07/23', '2022/07/24', '2022/07/30', '2022/07/31', '2022/08/06', '2022/08/07', '2022/08/13', '2022/08/14', '2022/08/20', '2022/08/21', '2022/08/27', '2022/08/28', '2022/09/03', '2022/09/04', '2022/09/05', '2022/09/10', '2022/09/11', '2022/09/17', '2022/09/18', '2022/09/24', '2022/09/25', '2022/10/01', '2022/10/02', '2022/10/08', '2022/10/09', '2022/10/15', '2022/10/16', '2022/10/22', '2022/10/23', '2022/10/29', '2022/10/30', '2022/11/05', '2022/11/06']
# def f(mo):       
#     mo_logs = vip_logs[vip_logs.order_number == mo].copy()
#     mo_logs = mo_logs.drop_duplicates(subset=['sequence_code'], keep='first')
#     mo_logs['diff'] = mo_logs['time_out'].diff(periods=1).astype('timedelta64[m]')
#     mo_logs['start'] = mo_logs['time_out'].shift(1)    
#     mo_logs.loc[mo_logs['diff'] > THROTTLE_MIN, 'diff'] = THROTTLE_MIN
#     try:
#         # for index, row in mo_logs.iloc[1:].iterrows():                
#         #     if row['diff'] > THROTTLE_MIN:
#         #         end = row.time_out.strftime("%Y-%m-%d")
#         #         start = row.start.strftime("%Y-%m-%d")
#         #         offday_count = len(list(x for x in low_days if start < x < end))             
#         #         row['diff'] -= offday_count * 1440
#         #         if row['diff'] > THROTTLE_MIN:
#         #             # print(index)
#         #             mo_logs.loc[index, 'diff'] = THROTTLE_MIN    
#         period = mo_logs['diff'].sum() / 60
#         # return mo, period, mo_logs.iloc[0]['time_out']
#         return mo, period, len(mo_logs)
#     except Exception as e: print(mo, e)